## Logistic Version of the Isolet Model Test in Wallaroo

The following demonstrates how to use a converted sk-learn Logistics ML model converted to ONNX  in Wallaroo.  This is meant to be a follow up to the tutorial `convert-sklearn-logistic-to-onnx.ipyb`, which demonstrates how to convert the sk-learn logicistics model to ONNX format and generates the file `isolet_logistic_model_numclass.onnx` that is used in this tutorial.

For more details on using Wallaroo workspaces, pipelines, and models, see the [Wallaroo Documentation site](https://docs.wallaroo.ai).

This tutorial includes the following assets:

* `isolet_test_data.tsv`:  A test file that can be used to verify the output of the converted logistic model.
* `test-converted-sklearn-logistics-to-onnx.ipynb`: This Jupyter Notebook that demonstrates the use of the converted sk-learn logistic ML model in ONNX with Wallaroo.

## Test Steps

### Load Libraries

The first step in the process is to load the necessary libraries.

In [1]:
# from their artifacts

import json
import wallaroo
import numpy
import pandas

from wallaroo.object import EntityNotFoundError

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

### Arrow Support

As of the 2023.1 release, Wallaroo provides support for dataframe and Arrow for inference inputs.  This tutorial allows users to adjust their experience based on whether they have enabled Arrow support in their Wallaroo instance or not.

If Arrow support has been enabled, `arrowEnabled=True`. If disabled or you're not sure, set it to `arrowEnabled=False`

The examples below will be shown in an arrow enabled environment.

In [2]:
import os
arrowEnabled=True
os.environ["ARROW_ENABLED"]=f"{arrowEnabled}"


### Connect to Wallaroo Instance

Next we'll connect to the Wallaroo Instance with the `Client` method.

In [ ]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

# SSO login through keycloak

# wallarooPrefix = "YOUR PREFIX"
# wallarooSuffix = "YOUR PREFIX"

# wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
#                     auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
#                     auth_type="sso")

### Connect to Workspace

Either create or use an existing workspace.  In this example, we will connect to the existing workspace `isolettest` by using the `set_current_workspace` command.  For more details on connecting to a workspace, see the [Wallaroo SDK Essentials guide](https://docs.wallaroo.ai/wallaroo-sdk/wallaroo-sdk-essentials-guide/).

![Wallaroo Workspace Example](./images/wallaroo-tutorials/isolet-workspace-example.png)

In [7]:
workspace_name = 'isolettest'
pipeline_name = 'isoletpipeline'
model_name = 'isolettest'

In [8]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(pipeline_name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(pipeline_name)
    return pipeline

workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

isoletpipeline = get_pipeline(pipeline_name)
isoletpipeline

name,isoletpipeline
created,2023-02-21 21:24:34.687987+00:00
last_updated,2023-02-21 21:24:34.687987+00:00
deployed,(none)
tags,
versions,5a346b73-750d-4059-a3d3-2b6803ed1e1e
steps,


### Upload Model

Upload the `isolet_logistic_model_numclass.onnx` model as `isolet-test` with the `upload_model` method:

In [9]:
model = wl.upload_model(model_name, './isolet_logistic_model_numclass.onnx').configure()

In [10]:
isoletpipeline.add_model_step(model)

name,isoletpipeline
created,2023-02-21 21:24:34.687987+00:00
last_updated,2023-02-21 21:24:34.687987+00:00
deployed,(none)
tags,
versions,5a346b73-750d-4059-a3d3-2b6803ed1e1e
steps,


### Deploy Pipeline

Pipelines can be deployed either with the Wallaroo Dashboard user interface or through the SDK.  For this example, we have the pipeline `isolet-pipeline` deployed and assigned to the variable `isoletpipeline`.  For more details, see the [Wallaroo Pipeline Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-pipeline-management/).

![Wallaroo Workspace Example](./images/wallaroo-tutorials/isolet-pipeline-example.png)

We can check the status of the pipeline with the Wallaroo pipeline `status` method.

In [138]:
isoletpipeline.deploy()

name,isoletpipeline
created,2023-02-21 21:24:34.687987+00:00
last_updated,2023-02-21 23:34:40.975970+00:00
deployed,True
tags,
versions,"7eaf3d89-91d8-4b74-a096-d5ca4a01c8bf, 90eaf9ec-29f6-4723-8999-d09ab82b18fe, 5a346b73-750d-4059-a3d3-2b6803ed1e1e"
steps,isolettest


### Load the Data

For Arrow enabled environments, we'll directly input the isolet test data into the pipeline.

For Arrow disabled environments, we'll load the data from the file `isolet_test_data.tsv`.

In [129]:
# read in data
testdata = pandas.read_csv('isolet_test_data.tsv', sep='\t')

We'll display the data shape to verify it has 9 rows and 617 columsn per row.

In [135]:
# report the size of the data  -  9 rows, 617 columns each
testdata.shape

(9, 617)

### Run Inference

Use the Wallaroo pipeline `infer` method to run an inference and store the results into the variable `result`, then display the data from the result.

In [144]:
if arrowEnabled is True:
    # convert to a tensor dataframe
    inputdata = pd.DataFrame(columns=['tensor'])
    for index, row in testdata.iterrows():
        inputdata = inputdata.append({'tensor': row.values.tolist()}, ignore_index=True)
    result = isoletpipeline.infer(inputdata)
    display(result['out.probabilities'])
else:
    # convert data into Wallaroo tensor JSON
    input_dict = {
        'tensor': testdata.values.tolist()
    }
    result = isoletpipeline.infer(input_dict)
    display(result[0].raw['outputs'])

0     [0.999963, 3.7014484e-05]
1     [0.99774987, 0.002250135]
2      [0.004735887, 0.9952641]
3      [0.031078666, 0.9689213]
4    [0.9996307, 0.00036931038]
5     [0.9992027, 0.0007972717]
6       [0.14781258, 0.8521874]
7     [0.080456555, 0.91954345]
8     [0.9987874, 0.0012125671]
Name: out.probabilities, dtype: object

### Closing

Once the tests are done, the pipeline should be undeployed to allocate the resources back to the Wallaroo instance.  For more details, see the [Wallaroo Pipeline Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-pipeline-management/).

In [145]:
isoletpipeline.undeploy()

name,isoletpipeline
created,2023-02-21 21:24:34.687987+00:00
last_updated,2023-02-21 23:34:40.975970+00:00
deployed,False
tags,
versions,"7eaf3d89-91d8-4b74-a096-d5ca4a01c8bf, 90eaf9ec-29f6-4723-8999-d09ab82b18fe, 5a346b73-750d-4059-a3d3-2b6803ed1e1e"
steps,isolettest
